<br></br>
# **gensim | word2vec**
<br></br>
## **1 데이터 전처리**

In [12]:
import glob
from txtutil import txtnoun
# 2015 ~ 2018 지속가능 경영 보고서 Token을 수집
filelist = glob.glob('./data/kr-Report_201?.txt')
print(filelist)

# 불러온 Document 명사Token만 추출
skiplist = {'갤러시':'갤럭시', '가치창출':'가치창출'}
texts    = [txtnoun(file, skip=skiplist)  for file in filelist]
texts    = " ".join(texts)
texts[:300]

['./data/kr-Report_2016.txt', './data/kr-Report_2017.txt', './data/kr-Report_2015.txt', './data/kr-Report_2018.txt']


'삼성전자\n 지속가능경영보고서\n 삼성전자 지속가능경영보고서\n 삼성전자 지속가능경영 사람과 사회 환경 조화롭\n 공존\n 가치 컬러 그래픽 조화롭 심플 톤앤매너\n 표지 또한 인재 기술 바탕 최고 제품\n 서비스 창출하 인류사회 공헌 경영이념 사람 사회\n 환경 포괄 영역 삼성전자 활동 세분 라인 조합\n 형태 움직임 통해\n 보고서 작성 개요\n 삼성전자 경제 가치 환경 사회 발전 지속가능경영 통해 세상 긍정\n 가치 지속가능경영 활동 성과 공개 바탕\n 이해관계자 소통 위해 아홉 지속가능경영보고서 발간\n 보고기간 일부 성과 포함\n 보고범위 국내외 사'

In [ ]:
# 명사 Token 작업된 자료를 ssResport.txt 로 저장 
texts_file = './data/ssResport.txt'
with open(texts_file, 'w', encoding='utf-8') as file:
    file.write(texts)

In [ ]:
# ! cat ./data/ssResport.txt | head -n 10

<br></br>
## **2 Word 2 vec 객체 만들기**
gensim

In [ ]:
# ! pip3 install gensim

In [ ]:
%%time
texts_file = './data/ssResport.txt'

from gensim.models import word2vec
data  = word2vec.LineSentence(texts_file)
model = word2vec.Word2Vec(data, size=200, window=2, min_count=20, hs=1,
                          workers=4, iter=100, sg=1)
model.save("./data/ssReport.model")
print("model saved.")

<br></br>
## **3 저장된 객체 활용**
gensim

In [ ]:
%reset

In [ ]:
%who

In [ ]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('./data/ssReport.model')
len(model.wv.vocab.keys())

In [ ]:
list(model.wv.index2word)[:10]

In [ ]:
model.wv.most_similar(positive=['삼성전자'])

In [ ]:
model.wv.most_similar(negative=['삼성전자'])

In [ ]:
model.wv.most_similar(positive=['글로벌'])

In [ ]:
model.wv.most_similar(negative=['글로벌'])

In [ ]:
model.wv.most_similar(positive=['삼성전자','경영활동'], 
                      negative=['근무환경']) # 담당자, 직원

<br></br>
## **04 Visulaization**
gensim

In [ ]:
list(model.wv.vocab.keys())[:10]

In [ ]:
# model.wv.vocab : { word: object of numeric vector }
vocab  = list(model.wv.vocab)
X      = model[vocab]
X.shape

In [ ]:
%%time
from sklearn.manifold import TSNE
tsne   = TSNE(n_components= "=Quiz!=")
X_tsne = tsne.fit_transform(X)

In [ ]:
import pandas as pd
df = pd.DataFrame(X_tsne, 
                  index = vocab, 
                  columns=['x', 'y'])
df.head()

In [ ]:
%matplotlib inline
from matplotlib import rc
rc('font', family=['NanumGothic','Malgun Gothic'])

import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12,12))
ax  = fig.add_subplot(1, 1, 1)
ax.scatter(df['x'], df['y'])
for word, pos in df.iterrows():
    ax.annotate(word, pos)
plt.grid(True)